In [ ]:
import sys
if 'google.colab' in sys.modules:  # Google Colab environment
    # Installing requisite packages
    !pip install datasets transformers==4.37.2 evaluate accelerate optimum auto-gptq

    # Mount google drive to enable access to data files
    from google.colab import drive
    drive.mount('/content/drive')

# Adjust the path below according to the actual location of your file within Google Drive
file_path = '/content/drive/My Drive/VSM_BRIMS_03_02.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

In [ ]:
#load data
df = pd.read_csv('/content/drive/My Drive/VSM_BRIMS_03_02.csv', header=0)

# Manually specify column names if necessary
df.columns = ['task', 'participant', 'trial', 'decision_type', 'choice', 'OEE1', 'OEE2', 'CT1', 'CT2']

print(df.head())
df['multiclass_target'] = df['choice'] * 3 + df['decision_type']
print(df.head())

   task  participant  trial  decision_type  choice  OEE1  OEE2  CT1  CT2
0     0            0      0              0       1    88    86   46   48
1     0            0      1              1       1    88    86   46   48
2     0            0      2              0       1    88    86   46   48
3     0            0      3              0       1    88    86   46   48
4     0            0      4              0       1    88    86   46   48
   task  participant  trial  decision_type  choice  OEE1  OEE2  CT1  CT2  \
0     0            0      0              0       1    88    86   46   48   
1     0            0      1              1       1    88    86   46   48   
2     0            0      2              0       1    88    86   46   48   
3     0            0      3              0       1    88    86   46   48   
4     0            0      4              0       1    88    86   46   48   

   multiclass_target  
0                  3  
1                  4  
2                  3  
3            

In [ ]:
if 'task' in df.columns:
    print("Task column is present.")
else:
    print("Task column is missing. Available columns:", df.columns)

Task column is present.


In [ ]:
# iterative prompts include strategy used by VSM-ACTR explained
import pandas as pd

# question template
question_template = (
   "Our manufacturing line features two sections prone to defects: pre-assembly (0) and assembly (1)."
"Pre-assembly requires {CT1} seconds to complete with an Overall Equipment Effectiveness (OEE) rate of {OEE1}%."
"Assembly takes {CT2} seconds and has an OEE rate of {OEE2}%."
"To cut total assembly time by 4 seconds, we must decide which section's duration can be reduced with the least increase in defects."
"Reducing cycle times will also result in higher headcount costs."
"We have three strategy levels for decision-making:"
"- Novice strategy (targets encoded as 0 for pre-assembly, 3 for assembly): Intuitive choice."
"- Intermediate strategy (targets encoded as 1 for pre-assembly, 4 for assembly): Make decision using key metrics"
"- Expert strategy (targets encoded as 2 for pre-assembly, 5 for assembly): make well-informed judgments based on a comprehensive understanding of all relevant metric."
"Question: Given the different strategy levels, which options would you choose?"
"Answer: "
)

text = []

# Iterate over each task
for task in df['task'].unique():
    df_task = df[df['task'] == task]
    print(task)
    if not df_task.empty:
        OEE1 = df_task['OEE1'].iloc[0]
        OEE2 = df_task['OEE2'].iloc[0]
        CT1 = df_task['CT1'].iloc[0]
        CT2 = df_task['CT2'].iloc[0]
        prompt = question_template.format(OEE1=OEE1, OEE2=OEE2, CT1=CT1, CT2=CT2)

        # Apply the prompt to each row in the task
        for index, row in df_task.iterrows():
            text.append(prompt)
    else:
        # Assuming each task should at least have one entry in 'text'
        num_trials_expected = 15  # Default number of trials if it's a fixed number per participant
        text.extend(["Data not available for this task."] * num_trials_expected)

# Adjust the list size to match the DataFrame in case of any mismatches
if len(text) < len(df):
    # Add placeholder texts if 'text' is shorter
    text.extend(["Data missing due to processing error."] * (len(df) - len(text)))
elif len(text) > len(df):
    # Trim 'text' if it's longer
    text = text[:len(df)]

# Add the generated text as a new column in the DataFrame
df['text'] = text


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [ ]:
dat = Dataset.from_pandas(df)
dat

Dataset({
    features: ['task', 'participant', 'trial', 'decision_type', 'choice', 'OEE1', 'OEE2', 'CT1', 'CT2', 'multiclass_target', 'text'],
    num_rows: 2012
})

In [ ]:
dat[0]

{'task': 0,
 'participant': 0,
 'trial': 0,
 'decision_type': 0,
 'choice': 1,
 'OEE1': 88,
 'OEE2': 86,
 'CT1': 46,
 'CT2': 48,
 'multiclass_target': 3,
 'text': "Our manufacturing line features two sections prone to defects: pre-assembly (0) and assembly (1).Pre-assembly requires 46 seconds to complete with an Overall Equipment Effectiveness (OEE) rate of 88%.Assembly takes 48 seconds and has an OEE rate of 86%.To cut total assembly time by 4 seconds, we must decide which section's duration can be reduced with the least increase in defects.Reducing cycle times will also result in higher headcount costs.We have three strategy levels for decision-making:- Novice strategy (targets encoded as 0 for pre-assembly, 3 for assembly): Intuitive choice.- Intermediate strategy (targets encoded as 1 for pre-assembly, 4 for assembly): Make decision using key metrics- Expert strategy (targets encoded as 2 for pre-assembly, 5 for assembly): make well-informed judgments based on a comprehensive under

In [ ]:
# Importing the necessary class from transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load model
model_ckpt = 'TheBloke/LLama-2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(
    model_ckpt,
    device_map="auto",
    revision="main"
)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True)

print (model.config.to_json_string())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/913 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at TheBloke/LLama-2-13B-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.10.self_attn.v_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11.mlp.gate_proj.

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

{
  "_name_or_path": "TheBloke/LLama-2-13B-GPTQ",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 13824,
  "max_length": 4096,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 40,
  "num_hidden_layers": 40,
  "num_key_value_heads": 40,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "batch_size": 1,
    "bits": 4,
    "block_name_to_quantize": null,
    "cache_block_outputs": true,
    "damp_percent": 0.01,
    "dataset": null,
    "desc_act": false,
    "exllama_config": {
      "version": 1
    },
    "group_size": 128,
    "max_input_length": null,
    "model_seqlen": null,
    "module_name_preceding_first_block": null,
    "modules_in_block_to_quantize": null,
    "pad_token_id": null,
    "quant_method": "gptq",
    "sym": tr

In [ ]:
!pip install peft

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
batch_tokenizer = lambda batch: tokenizer(batch['text'], padding=True, truncation=True)

#  Tokenizing the dataset
dat = dat.map(batch_tokenizer, batched=True)
dat[0]

Map:   0%|          | 0/2012 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'task': 0,
 'participant': 0,
 'trial': 0,
 'decision_type': 0,
 'choice': 1,
 'OEE1': 88,
 'OEE2': 86,
 'CT1': 46,
 'CT2': 48,
 'multiclass_target': 3,
 'text': "Our manufacturing line features two sections prone to defects: pre-assembly (0) and assembly (1).Pre-assembly requires 46 seconds to complete with an Overall Equipment Effectiveness (OEE) rate of 88%.Assembly takes 48 seconds and has an OEE rate of 86%.To cut total assembly time by 4 seconds, we must decide which section's duration can be reduced with the least increase in defects.Reducing cycle times will also result in higher headcount costs.We have three strategy levels for decision-making:- Novice strategy (targets encoded as 0 for pre-assembly, 3 for assembly): Intuitive choice.- Intermediate strategy (targets encoded as 1 for pre-assembly, 4 for assembly): Make decision using key metrics- Expert strategy (targets encoded as 2 for pre-assembly, 5 for assembly): make well-informed judgments based on a comprehensive under

In [ ]:
dat[0]

{'task': 0,
 'participant': 0,
 'trial': 0,
 'decision_type': 0,
 'choice': 1,
 'OEE1': 88,
 'OEE2': 86,
 'CT1': 46,
 'CT2': 48,
 'multiclass_target': 3,
 'text': "Our manufacturing line features two sections prone to defects: pre-assembly (0) and assembly (1).Pre-assembly requires 46 seconds to complete with an Overall Equipment Effectiveness (OEE) rate of 88%.Assembly takes 48 seconds and has an OEE rate of 86%.To cut total assembly time by 4 seconds, we must decide which section's duration can be reduced with the least increase in defects.Reducing cycle times will also result in higher headcount costs.We have three strategy levels for decision-making:- Novice strategy (targets encoded as 0 for pre-assembly, 3 for assembly): Intuitive choice.- Intermediate strategy (targets encoded as 1 for pre-assembly, 4 for assembly): Make decision using key metrics- Expert strategy (targets encoded as 2 for pre-assembly, 5 for assembly): make well-informed judgments based on a comprehensive under

In [ ]:
dat.set_format('torch', columns=['input_ids', 'attention_mask'])
dat

Dataset({
    features: ['task', 'participant', 'trial', 'decision_type', 'choice', 'OEE1', 'OEE2', 'CT1', 'CT2', 'multiclass_target', 'text', 'input_ids', 'attention_mask'],
    num_rows: 2012
})

In [ ]:
import torch
torch.manual_seed(42) # For reproducibility
from transformers import AutoModel

In [ ]:
# Loading the model and moving it to the GPU if available
if torch.cuda.is_available():  # for nvidia GPUs
    device = torch.device('cuda')
elif torch.backends.mps.is_available(): # for Apple Metal Performance Sharder (mps) GPUs
    device = torch.device('mps')
else:
    device = torch.device('cpu')

device

device(type='cuda')

# Feature extraction for decision making prediction

In [ ]:
#feature extraction
def extract_features(batch):
    inputs = {
        k:v.to(device)for k, v in batch.items() if k in tokenizer.model_input_names
    }
    with torch.no_grad():
        last_hidden_state = model(
            **inputs,
            output_hidden_states=True
        ).hidden_states[-1]
    return {
        "hidden_state": last_hidden_state[:, -1].cpu().numpy()
    }

In [ ]:
dat = dat.map(extract_features, batched=True, batch_size=1)
dat['hidden_state'].shape

Map:   0%|          | 0/2012 [00:00<?, ? examples/s]

torch.Size([2012, 5120])

In [ ]:
features = pd.DataFrame(dat['hidden_state'])
features

features.to_csv('features_2.csv', index=False)  # Saving as CSV without the index

## prediction for decision making results and strategy

In [ ]:
file_path2 = '/content/drive/My Drive/features_2.csv'
features = pd.read_csv(file_path2)

from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, log_loss
import pandas as pd
# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, dat['multiclass_target'], test_size=0.2, random_state=42)
print(f'Train size: {len(X_train)}, test size: {len(X_test)}')

# Standardizing the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Setting up the LogisticRegressionCV model for multiclass classification with 10-fold cross-validation
clf = LogisticRegressionCV(cv=10, multi_class='multinomial', max_iter=1000)

# Fitting the model
clf.fit(X_train, y_train)

# Predicting the test set results
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)  # Get the probability estimates for the test set

# Evaluating performance using different metrics
accuracy = accuracy_score(y_test, y_pred)
nll = log_loss(y_test, y_proba)  # Calculate Negative Log-Likelihood

# Printing out the results
print(f'Accuracy = {accuracy}')

print(f'Negative Log-Likelihood = {nll}')


Train size: 1609, test size: 403


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Accuracy = 0.4218362282878412
Negative Log-Likelihood = 1.177992551589777
